# ASLM parameters optimization

Optimize acquisition parameters of mesoSPIM: 
- $t_{exp}$, pixel exposure time, typically 5-20 ms
- $f_{gal}$, galvo scanning frequency, typically 99.99 or 199.9 Hz. Lower is preferred due to heating issues.
    * Galvo one-way sweep time (half-cycle): $t_{gal} = 0.5 * 1000/f_{gal}$ ms, typically 5 or 2.5 ms.

### Experimental setup
Benchtop mesoSPIM with 5X objective and Iris 15 camera, $FOV_X = 2522$ um, rolling shutter mode (pixels are read out by columns with defined exposure time per pixel).

The measured gaussian beam (488 nm) sigma along propagation axis is $\sigma=85$ um, so $L_{beam}=2\sigma = 170$ um. Strictly speaking, LS length is $FWHM_z = 2.355\sigma = 2 Z_R$, two Rayleigh lengths. But using $2\sigma$ is more conservative and should yield somewhat thinner LS.

### Optimization assumptions
1. Each pixel is illuminated via galvo sweep one or more times, : $n=1,2,...$, depending on $t_{exp}$ setting.
2. During $t_{exp}$ interval, laser waist travels some distance
    * beam waist speed $V_{beam}=FOV_X/T_{etl-ramp}$
    * ideally, beam waist should travel no more than $L_{beam}$ distance during $t_{exp}$ interval.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [204]:
# Optics and sensor params
MAG = np.array([2, 5, 7.5, 10])
FOV_X_sensor_um = 12610 # Iris 15 shorter side
FOV_X_um = FOV_X_sensor_um / MAG
ETL_ramp_duty_ratio = 0.85 # default 0.85
f_gal = 99.9 # user-defined parameter, galvo frequency
t_exp_ms = 10 # user-defined parameter, pixel exposure time
T_waveform_ms = 250 # user-defined parameter, total waveform duration, default 250
L_beam_um = 120 # conservative estimate

In [210]:
T_ETL_ramp_ms = T_waveform_ms * ETL_ramp_duty_ratio # active time of ETL ramp
t_gal = 0.5 * 1000/f_gal # half-cycle of the galvo, single sweep
V_beam = FOV_X_um / T_ETL_ramp_ms # physical speed of the beam waist along the beam axis
t_exp_max_ms = L_beam_um/V_beam # maximum recommended exposure time, based on beam length and waist speed
n_sweeps_per_max_exp = t_exp_max_ms/t_gal # how many galvo sweeps contained in the maximum recommended exposure time
n_sweeps_per_curr_exp = t_exp_ms/t_gal # how many galvo sweeps contained in actual user-selected exposure time
#hint = '<' if t_exp_ms < t_exp_max_ms else 'exceeds (!)'

print(f"Estimated beam length, um: {L_beam_um}")
print(f"Galvo freq, Hz: {f_gal}")
print(f"Galvo single sweep time, ms: {t_gal:.2} (= minimal recommended exposure)")
# print(f"Beam waist speed, um/ms: {V_beam:.2f}")
# print(f"Current exposure, ms: {t_exp_ms:.2f} {hint} maximum allowed {t_exp_max_ms:.2f}")
print(f"User-defined exposure, ms: {t_exp_ms:.1f}")
print(f"# galvo sweeps during user-defined exposure time ({t_exp_ms:.1f} ms): {t_exp_ms/t_gal:.1f}")
# print(f"# of beam sweeps per current exposure: {n_sweeps_per_curr_exp:.1f} {hint} maximum allowed {n_sweeps_per_max_exp:.1f}")
opt_params = pd.DataFrame({'MAG': MAG,  't_exp_max_ms': t_exp_max_ms, 'n_sweeps_per_max_exp': n_sweeps_per_max_exp})
opt_params.T

Estimated beam length, um: 120
Galvo freq, Hz: 99.9
Galvo single sweep time, ms: 5.0 (= minimal recommended exposure)
User-defined exposure, ms: 10.0
# galvo sweeps during user-defined exposure time (10.0 ms): 2.0


,0,1,2,3
MAG,2.000000,5.000000,7.500000,10.000000
t_exp_max_ms,4.044409,10.111023,15.166534,20.222046
n_sweeps_per_max_exp,0.808073,2.020182,3.030274,4.040365


# Legend
- `t_exp_max_ms` is the maximum recommended exposure time, based on beam length and waist speed, see notes below.
- `n_sweeps_per_max_exp` is how many galvo sweeps occur within the one `t_exp_max_ms` (should be at least 1)

# Notes
- exposure higher than `t_exp_max_ms` worsens the axial resolution by using wider-than-optimal LS length (with increasingly widening LS lobes)

- exposure smaller than `t_exp_max_ms` uses narrower LS length => somewhat higher z-resolution (10-20%?), but dark vertical stripes may apper due to missing galvo sweeps during a short exposure.

- it is better to guarantee at least 2 galvo sweeps per pixel (instead of ~1 or less), because non-sufficient sweep coverage creates dark stripes.

- higher magnification => higher recommended `t_exp_max_ms`

- 199.9 Hz galvo freq is generally no better than 99.9 Hz in resolution, but allows more galvo sweeps per exposure => better averaging, less striping. 199.9 Hz freq generates a lot of heat at 2X (large amplitude), but tolerable at 5X or more.

- 2X has challenging scanning parameters - FOV is large, so the exposure of 10 ms is suboptimal (thicker LS sectioning). However, XY resolution of 2X is 5 um at best, so widening of axial resolution to the same value is OK (isotropic resolution).

- the recommended exposure time for 2x-10X is **10 ms or higher**.

- the recommended galvo frequency for 2x-10s is **99.9Hz**

- the recommended total sweep (waveform) time is **250 ms**.


# Troubleshooting
## Dark vertical stripes
The vertical stripes appear when pixel exposure too low, galvo frequency too low, or both.
- increase exposure (5=>10. 15 ms, etc)
- increase galvo frequency (99=>199)

## Low (or fluctuating high/low) axial resolution
- decrease exposure (15 => 10, 5 ms, but not below 5 ms)

## Dim signal (low SNR image)
- increase the laser power
- double both the total waveform time and the exposure time (slows the acquisition speed by 2X!)
    - double only the exposure time, if you can tolerate worsening of axial resolution but want to keep the same acquisition speed